# Input data and libraries

In [1]:
# Input libraries
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# import pandapower as pp

In [2]:
# MEF
MEF = pd.read_csv("./source_data/CA_LR_24h.csv",index_col = 0)['MEF'].tolist()

In [3]:
# EV charging data from a workplace in the whole year of 2019

# With an example of the scenario with 50 charging piles. The other available numbers are 10, 90, 130, 170, 210, 250, 290, 330, 370, 410, 450
EVC_workplace_annual_50 = pd.read_csv("./result/EVC/EVC_workplace_50.csv",index_col=0)

In [4]:
# based load of a workplace building with 30 floors from 2018 to 2020
BL_workplace_avg = pd.read_csv("./result/base_load/BL_workplace_annual_avg.csv",index_col=[0,1])
BL_workplace_25th = pd.read_csv("./result/base_load/BL_workplace_annual_25th.csv",index_col=[0,1])
BL_workplace_75th = pd.read_csv("./result/base_load/BL_workplace_annual_75th.csv",index_col=[0,1])

BL_workplace_2019_avg = BL_workplace_avg.loc[BL_workplace_avg.index.get_level_values(0) == 2019]
BL_workplace_2019_25th = BL_workplace_25th.loc[BL_workplace_25th.index.get_level_values(0) == 2019]
BL_workplace_2019_75th = BL_workplace_75th.loc[BL_workplace_75th.index.get_level_values(0) == 2019]

# The function to extract the EV charging session data for one day

In [6]:
# Effective dates on the EV charging dataset
# Label the working days and non-working days in 2019 in USA

start = datetime(2019,1,1)
end = datetime(2019,12,31)
weekday_index = pd.bdate_range(start, end)
weekday_list = weekday_index.strftime("%Y-%m-%d").tolist()
wholeyear_index = pd.date_range(start, end)
wholeyear_list = wholeyear_index.strftime("%Y-%m-%d").tolist()

from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holiday_index = cal.holidays(start = "2019-01-01", end = "2019-12-31")
holiday_list = holiday_index.strftime("%Y-%m-%d").tolist() 

wd_list = [i for i in weekday_list if i not in holiday_list]
nwd_list = [i for i in wholeyear_list if i not in wd_list]

No_data_wd = ["2019-12-05","2019-12-06","2019-12-09","2019-12-10","2019-12-31"]
wd_list_effective = [i for i in wd_list if i not in No_data_wd]
wd_list_effective.append("2019-10-14")
wd_list_effective.append("2019-11-11")

# 2019-10-14，2019-11-11大功率
# 2019-01-19，2019-05-27，2019-10-20负数
No_data_nwd = ["2019-01-19","2019-01-20",
               "2019-02-16",
               "2019-04-14","2019-04-28",
               "2019-05-25","2019-05-27",
               "2019-08-17",
               "2019-10-20",
               "2019-11-28","2019-11-30",
               "2019-12-07","2019-12-08","2019-12-22","2019-12-25"]

nwd_list_effective = [i for i in nwd_list if i not in No_data_nwd]
nwd_list_effective = [i for i in nwd_list_effective if i not in ["2019-10-14","2019-11-11"]]

date_linear = ["2019-01-01","2019-03-03","2019-08-31","2019-09-01","2019-12-26"]

# 点太少了，无法拟合出合适的函数，当作异常点处理:"2019-01-01"，"2019-03-03"
# 真的完全服从一条直线，指数函数无法拟合："2019-12-26"，"2019-08-31"，"2019-09-01"
# 服从指数函数的点："2019-01-27","2019-02-02","2019-02-18","2019-07-04","2019-07-05","2019-08-10","2019-08-18","2019-09-08"，
# "2019-10-06"，"2019-11-24","2019-11-29"

wholeyear_effective = [i for i in wholeyear_list if i not in No_data_nwd]
wholeyear_effective = [i for i in wholeyear_effective if i not in No_data_wd]
# len(wholeyear_effective)

In [7]:
# sample one day EV charging data of the workplace from the annual dataset
def sample_EVC_workplace_oneday(EVC_workplace,random_seed):
    """
    EVC_workplace: EV charging data in the workplace for the whole year,df
    random_seed: the value of the random seed for np.random.seed, float
    """
    # EV充电数据含有的日子
    Date_EVC_workplace = (EVC_workplace
                          ["connect_date"]
                          .unique()
                          .tolist())
    
    # EV充电数据含有的工作日、非工作日
    # Date_EVC_workplace_nwd = [i for i in Date_EVC_workplace if i in nwd_list_effective]
    Date_EVC_workplace_wd = [i for i in Date_EVC_workplace if i in wd_list_effective]

    # 随机抽取日期序列，并赋予日期到各个节点中
    np.random.seed(random_seed) # 1 30 40 
    Date_EVC_workplace = np.random.choice(Date_EVC_workplace_wd)
    EVC_workplace = EVC_workplace.query("connect_date == @Date_EVC_workplace").copy().reset_index(drop=True)
    # EVC_workplace.to_csv("./result/EVC/EVC_workplace.csv")
    # EVC_public.to_csv("./result/EVC/EVC_public.csv")
    
    return EVC_workplace 

#  Load flow analysis to calculate the load limit under the constraints on voltage deviation, transformer capacity, and electrical line transmission capacity

In [11]:
# build the grid network model in pandapower
def network_3bus_1tr(p,q,transfo_capacity):
    """
    # the network looks like:
    # ext_grid b1 ---- b2 trafo(10/0.4kV) b3 load

    p: the active power demand, MW, float
    q: the reactive power demand, MVar, float
    trafo_capacity: the sn_mva of the transformer, float
    """ 
    #create empty net
    net = pp.create_empty_network()

    #create buses
    b1 = pp.create_bus(net, vn_kv=10, name="Bus_0")
    b2 = pp.create_bus(net, vn_kv=10, name="Bus_1")
    b3 = pp.create_bus(net, vn_kv=0.4, name="Bus_2")

    #create bus elements
    pp.create_ext_grid(net, bus=b1, name="Grid Connection")
    pp.create_load(net, bus=b3, p_mw=p, q_mvar=q, name='load')

    #create branch elements
    pp.create_line(net,from_bus=b1,to_bus=b2,length_km=0.1,std_type='94-AL1/15-ST1A 10.0',name='line_01')
    if transfo_capacity == 0.63:
        pp.create_transformer(net,hv_bus=b2,lv_bus=b3,name='trafo_12',std_type='0.63 MVA 10/0.4 kV')
    if transfo_capacity == 0.50:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=1.2, pfe_kw=1, i0_percent=0.21, shift_degree=150, name='trafo_12')
    if transfo_capacity == 1.25:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.7, i0_percent=0.14, shift_degree=150, name='trafo_12')
    if transfo_capacity == 1.60:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.8, i0_percent=0.12, shift_degree=150, name='trafo_12')
    if transfo_capacity == 2.00:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.9, i0_percent=0.10, shift_degree=150, name='trafo_12')

    return net

In [12]:
# Obtain the maximum loading of the bus under the constraint on the voltage deviation, transformer loading and line loading. 
def func_p_site_max(transfo_capacity,q):
    """
    transfo_capacity: the sn_mva of the transformer, MVA. workplace: 1.25、1.60, public: 0.5、0.63
    q: the reactive load in the bus, MVar
    """
    # section 1: the hyperparameters of obtaining the maximum loading of each bus

    # maximum and minimum bus voltage magnitude unit
    v_maxpu = 1.05
    v_minpu = 0.95
    # the maximum percentage of the transformer loading
    trafo_loading_max = 100
    # the maximum percentage of the line loading
    line_loading_max = 100

    # section 2: conduct calculation

    # the number of charging sites
    num_site = 1
    # the number of bus
    num_bus = 3
    # the number of trafo
    num_trafo = 1
    # the number of line
    num_line = 1
    # Initialize the maximum loading of the bus
    p_site_max = [0 for i in range(num_site)]
    # Assume to enumerate the load by a step of 1e-2 from 0 to 6 mW
    p_values = np.arange(0,6,1e-2)
    for p in p_values: # 模拟每节点的load (mW)
        # run load flow analysis
        net = network_3bus_1tr(p,q,transfo_capacity)
        pp.runpp(net)

        # the voltage deviation of each bus
        v_pu = [net.res_bus.loc[i,'vm_pu'] for i in range(num_bus)]
        # the transformer loading(%)
        trafo_loading = [net.res_trafo.loc[i,'loading_percent'] for i in range(num_trafo)]
        # the line loading (%)
        line_loading = [net.res_line.loc[i,'loading_percent'] for i in range(num_line)]

        # A flag to indicate if any bus has the voltage deviation bigger than the limit. 
        # If yes, return True, the loop breaks; if no, return False, the loop continues.
        # the same for the transformer loading and line loading
        flag_v_pu = any(x < v_minpu or x > v_maxpu for x in v_pu)
        flag_trafo_loading = any(x > trafo_loading_max for x in trafo_loading)
        flag_line_loading = any(x > line_loading_max for x in line_loading)

        # If any flag is true, save the maximum power load for each bus then breaks the loop
        if any((flag_v_pu,flag_trafo_loading,flag_line_loading)):
            for i in range(num_site):
                p_site_max[i] = p
            break
    
    return p_site_max

In [13]:
# Conduct load flow analysis
%%capture
p_site_max_w1600 = func_p_site_max(transfo_capacity = 1.60,q = 0.10) # power factor was set to 0.9

# Functions to conduct online optimization of charging coordination under the computed load limit

In [8]:
# Optimization model 1：Obtain the upper bound of the load variance
def opt_model_ub(time,ce,p_max,delta_t,p_site_max,
                 site,d_t,EV_index,E,t_plugin,t_plugout,D_t,loadvar_ub):
    # Create optimization model
    m_ub = gp.Model('EVcharging_ub')

    # Create variables
    # 将字典EV转化为a list of tuple，第一位表示site,第二位表示EV的序号
    import itertools
    EV_list = [(k, v) for k, values in EV_index.items() for v in values]
    p_tbi = m_ub.addVars(time, EV_list, name="power") # time行 * num_EV列 * site列的矩阵
    p_t = gp.tuplelist([p_tbi.sum(t,"*","*") for t in time]) # 一个时刻所有site的所有EV的充电功率之和，time*1的向量
    p_bt = gp.tuplelist([[p_tbi.sum(t, site[0],"*") for t in time]])# 一个时刻一个site的所有EV充电功率之和（对EV求和），site*time的矩阵                
    p_tb = gp.tuplelist(list(map(list,zip(*p_bt)))) # 通过转置，变为time*site的矩阵
    p_bi = gp.tuplelist([[p_tbi.sum("*",site[0], i) for i in EV_index[site[0]]]])# 每个site的每辆EV的充电功率之和(对时间求和)

    # Set objective
    m_ub.setObjective (sum(np.multiply(p_t,MEF)) * delta_t / ce, GRB.MINIMIZE) # the output is g CO2 emissions after coordination

    # Add constraints
    for t in time:
        m_ub.addConstrs(
            (p_tbi[t,s,i] >= 0 for s in site for i in EV_index[s]),name="charging_power1")
        m_ub.addConstrs(
            (p_tbi[t,s,i] <= p_max for s in site for i in EV_index[s]),name="charging_power2")
        m_ub.addConstrs(
            (p_tb[t][s]/1_000 + d_t[s][t]/1_000 <= p_site_max[s] for s in site), name="site_max_loading")
    for s in site:
        for i in EV_index[s]:
            if t_plugin[s][i] < t_plugout[s][i]:
                if t_plugin[s][i] !=0 and t_plugin[s][i] != len(time)-1:
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(0,int(t_plugin[s][i]))),"charging_power_time1")
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,len(time))),"charging_power_time2")
                elif t_plugin[s][i] == 0:
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,len(time))),"charging_power_time1")
                else:
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(0,int(t_plugin[s][i]))),"charging_power_time1")
            else:
                if abs(t_plugin[s][i]-t_plugout[s][i])>1:
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,int(t_plugin[s][i]))),"charging_power_time1")
    m_ub.addConstrs(
        (ce * delta_t * p_bi[s][i] >= E[s][i] for s in site for i in EV_index[s]),"energy_demand")

    # Conduct optimization
#     m_ub.params.NonConvex = 1 # Quadratic equality constraints are non-convex. Set NonConvex parameter to 2 to solve model.
    m_ub.Params.MIPGap = 0.01    # termination criteria: 1%
    m_ub.Params.TimeLimit = 360  # termination criteria: 6 minutes
#     m_ub.Params.BarHomogeneous = 0 # turn off the homogeneous barrier algorithm
#     m_ub.Params.Method = -1 # automatic method = barrier
    m_ub.update()
    m_ub.optimize()
    if m_ub.status == GRB.OPTIMAL:
        solution_p_ub = m_ub.getAttr('x', p_tbi)
        loadvar_ub_value = sum((a + b) ** 2 for a, b in zip(D_t, [solution_p_ub.sum(t, '*','*').getValue() for t in time]))
        loadvar_ub.append(loadvar_ub_value)
    
    return m_ub.status

In [9]:
# Optimization model 2：Optimize under the constraint on the load varaince
def opt_model(time,ce,p_max,delta_t,p_site_max,Em_uc,loadvar_uc,total_energy,satisfied_energy_uc,
              site,d_t,EV_index,E,t_plugin,t_plugout,D_t,
              loadvar_ub,obj_result,var_result,random_seed):
    # Create optimization model
    m = gp.Model('EVcharging')

    # Create variables
    # 将字典EV转化为a list of tuple，第一位表示site,第二位表示EV的序号
    import itertools
    EV_list = [(k, v) for k, values in EV_index.items() for v in values]
    p_tbi = m.addVars(time, EV_list, name="power") # time行 * num_EV列 * site列的矩阵
    p_t = gp.tuplelist([p_tbi.sum(t,"*","*") for t in time]) # 一个时刻所有site的所有EV的充电功率之和，time*1的向量
    p_bt = gp.tuplelist([[p_tbi.sum(t, site[0],"*") for t in time]])# 一个时刻一个site的所有EV充电功率之和（对EV求和），site*time的矩阵                
    p_tb = gp.tuplelist(list(map(list,zip(*p_bt)))) # 通过转置，变为time*site的矩阵
    p_bi = gp.tuplelist([[p_tbi.sum("*",site[0], i) for i in EV_index[site[0]]]])# 每个site的每辆EV的充电功率之和(对时间求和)

    # Set objective
    m.setObjective (sum(np.multiply(p_t,MEF)) * delta_t / ce, GRB.MINIMIZE) # the output is g CO2 emissions after coordination

    # Add constraints
    for t in time:
        m.addConstrs(
            (p_tbi[t,s,i] >= 0 for s in site for i in EV_index[s]),name="charging_power1")
        m.addConstrs(
            (p_tbi[t,s,i] <= p_max for s in site for i in EV_index[s]),name="charging_power2")
        m.addConstrs(
            (p_tb[t][s]/1_000 + d_t[s][t]/1_000 <= p_site_max[s] for s in site), name="site_max_loading")
    for s in site:
        for i in EV_index[s]:
            if t_plugin[s][i] < t_plugout[s][i]:
                if t_plugin[s][i] !=0 and t_plugin[s][i] != len(time)-1:
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(0,int(t_plugin[s][i]))),"charging_power_time1")
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,len(time))),"charging_power_time2")
                elif t_plugin[s][i] == 0:
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,len(time))),"charging_power_time1")
                else:
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(0,int(t_plugin[s][i]))),"charging_power_time1")
            else:
                if abs(t_plugin[s][i]-t_plugout[s][i])>1:
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,int(t_plugin[s][i]))),"charging_power_time1")
    m.addConstrs(
        (ce * delta_t * p_bi[s][i] >= E[s][i] for s in site for i in EV_index[s]),"energy_demand")
    m.addConstr(
        (sum((a + b) ** 2 for a, b in zip(D_t, [p_tbi.sum(t,"*","*") for t in time])) <= loadvar_ub),name="load_variance")

    # Conduct optimization
#     m.params.NonConvex = 1 # Quadratic equality constraints are non-convex. Set NonConvex parameter to 2 to solve model.
    m.Params.MIPGap = 0.01    # termination criteria: 1%
    m.Params.TimeLimit = 360  # termination criteria: 6 minutes
#     m.Params.BarHomogeneous = 0 # turn off the homogeneous barrier algorithm
#     m.Params.Method = -1 # automatic method = barrier
    m.update()
    m.optimize()
    if m.status == GRB.OPTIMAL:
        solution_p = m.getAttr('x', p_tbi)
        # save the charging power as lists
        p_tb_cc_0 = []
        for t in time:
            p_tb_cc_0.append(solution_p.sum(t,0,'*').getValue())
        p_tb_cc = [a for a in p_tb_cc_0]
        E_cc = sum(p_tb_cc) * delta_t * ce # the delivered energy in coordinated charging
        Em_cc = (m.objVal)/(1_000 * E_cc) # 将碳排放数据从g转换为kg/kWh
        # obtain the reduction in load variance and emissions
        loadvar_cc = sum((a + b) ** 2 for a, b in zip(D_t, [solution_p.sum(t, '*','*').getValue() for t in time]))
        loadvar_reduce = ((loadvar_uc - loadvar_cc) / loadvar_uc) * 100
        Em_reduce = ((Em_uc - Em_cc) / Em_uc) * 100
        satisfied_energy = (E_cc / total_energy) * 100
        # store the result        
        obj_result.loc[random_seed,'Em_uc_kg/kWh'] = Em_uc # kg/kWh
        obj_result.loc[random_seed,'Em_cc_kg/kWh'] = Em_cc # kg/kWh
        obj_result.loc[random_seed,'Em_reduce_%'] = Em_reduce # %
        obj_result.loc[random_seed,'loadvar_uc'] = loadvar_uc
        obj_result.loc[random_seed,'loadvar_cc'] = loadvar_cc
        obj_result.loc[random_seed,'loadvar_reduce_%'] = loadvar_reduce # %
        obj_result.loc[random_seed,'satisfied_energy_%'] = satisfied_energy # %
        obj_result.loc[random_seed,'satisfied_energy_uc_%'] = satisfied_energy_uc # %
        var_result[random_seed] = p_tb_cc # kW

    return m.status

In [10]:
# Function to conduct charging coordination for the whole day
def func_opt(d_t_site_0,EVC_site_0,p_site_max,MEF,random_seed):
    """
    d_t_site_0: base load data in every 15 minutes, df
    EVC_site_0: EV charging session data, df
    p_site_max: the maximum loading of the bus under the constraint on the voltage deviation,transformer loading and line loading,list
    """
    # section 1: base data    
    
    # time interval (hour)
    delta_t = 0.25
    
    # number of time intervals of a day
    num_t = int(24*1/delta_t)
    time = [i for i in range(0,num_t)]

    # EV charging efficiency
    ce = 0.90

    # charging power limit of the L2 charger(kW)
    p_max = 14 # 10
    
    # coefficient of the uncoordinated EV charging load
#     alpha = [i/10_000 for i in range(10_000,8_992,-8)]
#     alpha = [i/10_000 for i in range(10_000,8_992,-80)]
    # alpha = 0.93

    # update the energy demand of each charging session by calculating the unit energy demand of each charging session
    for i in EVC_site_0.index: 
        EVC_site_0.loc[i,'kWhDelivered_unit'] = EVC_site_0.loc[i,'kWhDelivered'] / (EVC_site_0.loc[i,'disconnectTime_index'] - EVC_site_0.loc[i,'connectionTime_index'] + 1)                 
        if EVC_site_0.loc[i,"doneChargingTime_index"] > num_t - 1:
            EVC_site_0.loc[i,'kWhDelivered'] = EVC_site_0.loc[i,'kWhDelivered_unit'] * (num_t - EVC_site_0.loc[i,'connectionTime_index'] + 1) 
    
    total_energy = EVC_site_0['kWhDelivered'].sum()
    
    # the multidict indexed by the charging site
    site,d_t,EV_index,E,t_plugin,t_plugout = gp.multidict({
        0:[d_t_site_0.iloc[:,0].tolist(),
           range(EVC_site_0.shape[0]), # the index of EVs at each bus
           EVC_site_0['kWhDelivered'].tolist(), # the charging energy requirement of each EV at each bus, unit: kWh
           EVC_site_0['connectionTime_index'].tolist(),# the plug-in time of EVs at each bus (index of every 15 minutes)
           EVC_site_0['disconnectTime_index'].tolist()] # the plug-out time of EVs at each bus (index of every 15 minutes)
    })

    # total base load (kW)
    D_t = [a for a in d_t[0]]
    
    # section 2: Obtain the total load(kW), load variance, and CO2 emissions(kg) in uncoordinated charging
    
    # Obtain the charging demand (kW) at 15-min resolution
    def func_demand_load(EVC,num_t):
        """
        this func is used to calculate the charging load of a day in every 15 minutes
        param: 
        EVC: daily EV charging load, df
        num_t: number of time intervals of a day
        return:
        demand_load: the charging demand (kW) in every 15 minutes, list
        """
        # df for the total charging load in every 15 minutes(kW)
        demand_load = pd.DataFrame(data = [0 for i in range(0,num_t)])
        
        # copy EVC for revise
        df = EVC.copy()

        # load的第二列赋值，即计算某日EV充电负载，利用EV_charging_data按车作循环
        for i in df.index: # i是EV charging data的行，表示第i辆EV
            # 获得开始和结束的充电时间
            start_time = df.loc[i,"connectionTime_index"]
            end_time = df.loc[i,"doneChargingTime_index"] 
            power = df.loc[i,"charging_power"]
            if end_time >= num_t:
                for j in range(int(start_time),num_t,1):
                    demand_load.iloc[j,0] += power
            else:
                for j in range(int(start_time),int(end_time)+1,1):
                    demand_load.iloc[j,0] += power
        
        return demand_load[0].tolist()
    p_tb_demand = func_demand_load(EVC = EVC_site_0,num_t = num_t)
    p_tb_uc = [] # set up a list to store the power demand (kW) of uncoordinated charging at every 15-min
    for t in time:
        if p_tb_demand[t] + D_t[t] > p_site_max[0] * 1_000:
            p_tb_uc.append(p_site_max[0] * 1_000 - D_t[t])
        else:
            p_tb_uc.append(p_tb_demand[t])
    loadvar_uc = sum((a+b)**2 for a, b in zip(D_t,p_tb_uc))
    # calculate the energy demand fulfillment of uncoordinated charging
    satisfied_energy_uc = (sum(p_tb_uc) / sum(p_tb_demand)) * 100 # %  
    
    # Obtain the total CO2 emissions in uncooridinated charging
    def func_em(MEF_data,EV_charging_load,ce):
            """
            This function compute the CO2 emissions in uncoordinated charging    
            param 
            MEF_data: the MEF (g CO2/kWh) in every 15 mins of a day,list
            EV_charging_load: EV charging load (kW) before coordination in every 15 mins of a day, list
            ce: charging efficiency (%)
            return
            ucem: the total CO2 emissions in uncoordianted charging，array，unit: kg
            """
            # 利用两个向量内积，计算总排放量，充电效率设为0.85。时间由15分钟转换为小时，即乘以0.25
            ucem = (np.dot([0.25 * x for x in EV_charging_load],MEF_data)/ce)/1_000 # 将碳排放数据从g转换为kg

            return ucem
    Em_uc = func_em(MEF_data = MEF, EV_charging_load = p_tb_uc, ce = ce)/(sum(p_tb_uc) * delta_t * ce) # kg/kWh
                
    # Section 3: conduct charging coordination under the constraint of p_site_max
    
    # create two dfs to store the results
    # obj_result is for Em_uc,Em_cc,Em_reduce,loadvar_uc,loadvar_cc,loadvar_reduce, indexes are the series of alpha
    # var_result is for p_tb_cc. Columns are the indexes of random seeds, and indexes are the timeslots
    obj_result = pd.DataFrame(index = [random_seed], columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])
    var_result = pd.DataFrame(index = time, columns = [random_seed])
    
    # obtain the upper bound(largest) of the load variance
    loadvar_ub = []
    state_ub = opt_model_ub(time,ce,p_max,delta_t,p_site_max,site,d_t,EV_index,E,t_plugin,t_plugout,D_t,loadvar_ub)
    
    # derive the pareto front with different values on alpha. Break until the model is infeasible
    if state_ub == GRB.OPTIMAL:
        opt_model(time,ce,p_max,delta_t,p_site_max,Em_uc,loadvar_uc,total_energy,satisfied_energy_uc,
                  site,d_t,EV_index,E,t_plugin,t_plugout,D_t,
                  loadvar_ub[0],obj_result,var_result,random_seed)
                
    return obj_result,var_result

# Conduct the functions and output the result

In [17]:
# Define the DataFrames to store the outputs for the level 1 of annual existing electrical load

# For pareto front
df_pareto_w1600_50_25th = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of coordinated charging
df_p_tb_cc_w1600_50_25th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40,70)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50, random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_25th, EVC_site_0 = EVC_workplace_50,
                                                 p_site_max = p_site_max_w1600, MEF = MEF, random_seed = i)
    df_pareto_w1600_50_25th.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50_25th[i] = p_tb_cc_w1600_50

In [21]:
# Output the result
df_pareto_w1600_50_25th.to_csv("./result/pareto_ofcc/workplace/1600KVA/pareto_w1600_50_week_25th.csv")
df_p_tb_cc_w1600_50_25th.to_csv("./result/EV_charging_load_ofcc/workplace/1600KVA/p_tb_cc_w1600_50_week_25th.csv")

In [18]:
# Define the DataFrames to store the outputs for the level 2 of annual existing electrical load

# For pareto front
df_pareto_w1600_50 = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of coordinated charging
df_p_tb_cc_w1600_50 = pd.DataFrame(index = range(0,96),columns = [i for i in range(40,70)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50, random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_avg, EVC_site_0 = EVC_workplace_50,
                                                 p_site_max = p_site_max_w1600, MEF = MEF, random_seed = i)
    df_pareto_w1600_50.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50[i] = p_tb_cc_w1600_50

In [22]:
# Output the result
df_pareto_w1600_50.to_csv("./result/pareto_ofcc/workplace/1600KVA/pareto_w1600_50_week.csv")
df_p_tb_cc_w1600_50.to_csv("./result/EV_charging_load_ofcc/workplace/1600KVA/p_tb_cc_w1600_50_week.csv")

In [77]:
# Define the DataFrames to store the outputs for the level 3 of annual existing electrical load

# For pareto front
df_pareto_w1600_50_75th = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of coordinated charging
df_p_tb_cc_w1600_50_75th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40,70)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50, random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_75th, EVC_site_0 = EVC_workplace_50,
                                                 p_site_max = p_site_max_w1600, MEF = MEF, random_seed = i)
    df_pareto_w1600_50_75th.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50_75th[i] = p_tb_cc_w1600_50

In [81]:
# Output the result
df_pareto_w1600_50_75th.to_csv("./result/pareto_ofcc/workplace/1600KVA/pareto_w1600_50_week_75th.csv")
df_p_tb_cc_w1600_50_75th.to_csv("./result/EV_charging_load_ofcc/workplace/1600KVA/p_tb_cc_w1600_50_week_75th.csv")